<a href="https://colab.research.google.com/github/SeongGwan-a/Project-Classification-of-novelists/blob/main/CNN/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold


import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

import warnings 


warnings.filterwarnings(action='ignore')

# 데이터 전처리

In [ ]:
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')
#부호 제거 - 정규식 이용 
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# 해당 컬럼의 각 행에 위 함수 적용
train['text']=train['text'].apply(alpha_num)

In [ ]:
#불용어 제거

# 불용어

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)



In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# train test 분리하고 array로 structure 변경
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


# 모델링

In [ ]:
vocab_size = 20000
embedding_dim = 64 # 임베딩 차원
max_length = 500
padding_type='post'
n_class = 5

## vectorization

### one - hot encoding

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index # 각 토큰별로 인덱스 붙이기 

In [ ]:
# 각 행별로 토큰화된 단어들의 인덱스 리스트르 가짐
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
# 500길이로, 각 행의 뒤를 0으로 채움
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)

(54879, 500) (19617, 500)


### keras embedding +  CNN 모델 설정

In [ ]:
# 3번 cv 진행 
n_fold = 3
cv = StratifiedKFold(n_splits= n_fold, shuffle=True, random_state=42)

In [ ]:
def get_model():
    model = Sequential([
        #  무작위로 특정 차원으로 입력 벡터들을 뿌린 후 학습을 통해 가중치들을 조정해 나가는 방식 - 즉, 관계 반영하지 않음 
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(.5),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),    
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(.5),
        Dense(n_class, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.005))
    return model

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf = get_model()    
    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Train on 36584 samples, validate on 18295 samples
Epoch 1/10
36584/36584 [==============================] - 109s 3ms/sample - loss: 1.4041 - val_loss: 1.1157
Epoch 2/10
36584/36584 [==============================] - 103s 3ms/sample - loss: 0.9457 - val_loss: 0.8542
Epoch 3/10
36584/36584 [==============================] - 109s 3ms/sample - loss: 0.6819 - val_loss: 0.8240
Epoch 4/10
36584/36584 [==============================] - 111s 3ms/sample - loss: 0.5496 - val_loss: 0.8475
Epoch 5/10
36584/36584 [==============================] - 111s 3ms/sample - loss: 0.4542 - val_loss: 0.9040
Epoch 6/10
36584/36584 [==============================] - 110s 3ms/sample - loss: 0.3981 - val_loss: 0.9726
Epoch 00006: early stopping
training model for CV #2
Train on 36586 samples, validate on 18293 samples
Epoch 1/10
36586/36586 [==============================] - 113s 3ms/sample - loss: 1.4338 - val_loss: 1.2689
Epoch 2/10
36586/36586 [==============================] - 77s 2ms/

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  69.3963%
Log Loss (CV):   0.8309


In [ ]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

NameError: name 'p_val_file' is not defined

### 시각화

In [ ]:
# model summary
print(clf.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 64)           1280000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 165, 128)          57472     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 53, 128)           114816    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [ ]:
import pydot
plot_model(clf)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [ ]:
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
5,5,0,0,0,0,0
6,6,0,0,0,0,0
7,7,0,0,0,0,0
8,8,0,0,0,0,0
9,9,0,0,0,0,0


### 제출모델 생성

In [ ]:

sample_submission[sample_submission.columns[1:]] = p_tst
sample_submission.head()

,index,0,1,2,3,4
0,0,0.055973,0.175443,0.493678,0.109987,0.164919
1,1,0.074283,0.662041,0.052749,0.014820,0.196106
2,2,0.972492,0.014737,0.000160,0.000001,0.012610
3,3,0.089638,0.009645,0.804231,0.015374,0.081112
4,4,0.469599,0.060526,0.226578,0.057542,0.185755


In [ ]:
sample_submission.to_csv("subb.csv", index = False)

### word2vec 과 cnn

In [ ]:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)

ModuleNotFoundError: No module named 'gensim'